In [3]:
import pandas as pd
import pygsheets
import regex as re

In [4]:
frims = pd.read_csv(r'RawInputData/frims_damages.csv')
frims.crop_area_affected = frims.crop_area_affected.apply(lambda x: re.findall(r'\d+\.*\d*',x)[0])
frims.crop_area_affected = frims.crop_area_affected.astype(float)
frims_5y = frims.groupby(['object_id'])[['population_affected','crop_area_affected',
                                        'human_lives_lost','roads_dam','bridges_dam','embankment_total']].sum().reset_index()


In [5]:
procurement = pd.read_csv(r'RawInputData/procurement.csv')
cols = ['Count_Roads_Bridges_Embankments', 'Sum_Roads_Bridges_Embankments',
       'Count_Erosion', 'Sum_Erosion', 'Count_IM', 'Sum_IM', 'Count_new_const',
       'Sum_new_const', 'Count_repair_rest_imp', 'Sum_repair_rest_imp',
       'Count_relief_preparation', 'Sum_relief_preparation', 'Count_goods',
       'Sum_goods', 'Count_Total', 'Sum_Total', 'Count_SDRF', 'Sum_SDRF',
       'Count_SOPD', 'Sum_SOPD', 'Count_CIDF', 'Sum_CIDF', 'Count_LTIF',
       'Sum_LTIF', 'Count_RIDF', 'Sum_RIDF']
procurement_5y = procurement.groupby(['object_id'])[cols].sum().reset_index()

In [6]:
infra_access = pd.read_csv(r'RawInputData/infra_access.csv')
infra_access = infra_access.groupby('object_id')[['proximity_hosptial_rd', 'proximity_embankment_rd',
       'proximity_rail_rd', 'proximity_local_rd', 'proximity_arterial_rd',
       'ndbi']].median().reset_index()

In [7]:
demographicvulnerability = pd.read_csv(r'RawInputData/demographicvulnerability.csv')
demographicvulnerability = demographicvulnerability.groupby('object_id')[['ind_ppp_UNadj', 'aged', 'young', 'percaay',
       'deprived', 'nophone', 'noSanitation', 'nodrinkingWater',
       'totLivestock']].median().reset_index()

In [8]:
floodproneness = pd.read_csv(r'RawInputData/floodproneness.csv')

floodproneness = pd.get_dummies(floodproneness,
                                      columns = ['assam_soil', 'assam_lith','landuse'],
                                      dummy_na=False,
                                      drop_first=True)


cat_cols = ['assam_soil_loam, silt loam, silt, sandy loam',
       'assam_soil_loamy sand, sand',
       'assam_soil_rocky, other non-soil categories',
       'assam_lith_neogene sedimentary rock',
       'assam_lith_paleogene sedementary rock', 'assam_lith_paleozoic rock',
       'assam_lith_quaternary sediments',
       'assam_lith_tertiary sedimentary rocks',
       'assam_lith_undeveloped precambrian rock', 'assam_lith_water',
       'landuse_range land', 'landuse_vegetation', 'landuse_water','object_id']

floodproneness_cat = floodproneness[cat_cols].groupby('object_id').agg(lambda x:x.value_counts().index[0])
floodproneness_cat.columns = ['assam_soil_silt',
       'assam_soil_loamysand',
       'assam_soil_rocky',
       'assam_lith_neogene',
       'assam_lith_paleogene', 'assam_lith_paleozoic',
       'assam_lith_quaternary',
       'assam_lith_tertiary',
       'assam_lith_undevelopedprecambrian', 'assam_lith_water',
       'landuse_rangeland', 'landuse_vegetation', 'landuse_water']
floodproneness_con = floodproneness.groupby('object_id')[['gmted_drainage_density_without_1',
       'srtm_filled_dem', 'GCN250_ARCIII_average', 'Inundation', 'sum',
       'assam_dist_from_major_rivers_updated_3857',
       'strm_filled_slope_degrees', 'ndvi']].mean()

floodproneness = pd.concat([floodproneness_con,floodproneness_cat],axis=1).reset_index()

In [9]:
consolidated_5y = floodproneness.merge(demographicvulnerability,on='object_id').\
        merge(infra_access,on='object_id').merge(frims_5y,on='object_id').merge(procurement_5y,on='object_id',how='left')

In [10]:
consolidated_5y.to_csv('SEM_InputData/SEM_Input_Consolidated_5Years.csv',index=False)

In [11]:
google_client = pygsheets.authorize(service_file='idea-frm-500603810b1a.json')
SEM_Input_ConsolidatedRC_google_sheet = google_client.open('SEM_Input_Consolidated')

In [12]:
consolidated_5y_sheet = SEM_Input_ConsolidatedRC_google_sheet.worksheet('title','cum_5y')
consolidated_5y_sheet.clear()
consolidated_5y_sheet.set_dataframe(consolidated_5y, (1,1))